In [53]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
from math import gcd # greatest common divisor

register1 = 0
register2 = 0
a = 7


In [54]:
def setAmtQubits(N):
    T1 = N*N
    t1 = bin(T1)
    global register1 
    register1 = int(len(t1)-2)
    global register2
    register2 = math.ceil(math.log2(N))


In [57]:
setAmtQubits(43621)

print(register1)
print(register2)

31
16


In [160]:
def a2jmodN(a, j, N):
    """Compute a^{2^j} (mod N) by repeated squaring"""
    for i in range(j):
        a = np.mod(a**2, N)
    return a

In [166]:
print(a2jmodN(7,4,53))


28
